# Owner #1

## Initital set up

In [ ]:
#import packages that allow for image tranformations and transform to Tensor
from skimage.transform import resize
from skimage.color import rgb2gray
import torch
import os
import imageio
import cv2 as cv
from tqdm.notebook import tqdm

In [ ]:
#loop through the directory and transform to tensor
path = "/Users/danielcastillo/Documents/senior-project/Data/CATS_DOGS/test/CAT"

#transform the images to a list Tensors
transform(path)

In [ ]:
#Create a new duet session
import syft as sy
sy.VERBOSE = False
duet = sy.duet(loopback = True)

## Wait! Run Data Scientist till Checkpoint 1

In [ ]:
import tensorflow as tf
#change the list of tensors into a single tensor
stacked = torch.stack(arr)
#change the tensor data type from Float64 to Float32
stacked = stacked.type(torch.FloatTensor)

In [ ]:
#assign a tag to the image
image = stacked.tag("test cats")
#assign a description
image = stacked.describe("tensor data")

#send the pointer through duet
image_ptr = image.send(duet, searchable=True)

In [ ]:
#check all the pointers currently in the duet store
duet.store.pandas

In [ ]:
#add a hadler to accept any request with the name "loss"
duet.requests.add_handler(
    name="loss",
    action="accept",
    timeout_secs=-1,  # no timeout
    print_local=True  # print the result in your notebook
)

# Definition for transform()

In [ ]:
def transform(path):
    #declare a new list
    arr = []
    
    tqdm().pandas()
    i = 0


    for pic in tqdm(os.scandir(path)):
        #control the number of images to transform
        if i < 100:
            #check if file is of valid type 
            if (pic.path.endswith(".jpeg") or pic.path.endswith(".jpg")) and pic.is_file():
                #read the image
                picture = cv.imread(pic.path)
                if(type(picture) != None):
                    #resize the picture to 10 x 10
                    picture = resize(picture, (10,10), mode='constant', anti_aliasing=True)
                    #change the picture to gray scale
                    picture = rgb2gray(picture)
                    #transform the picture to a tensor
                    Tensor = torch.tensor(picture)
                    Tensor = Tensor.flatten()
                    #append the tensor to the list
                    arr.append(Tensor)
                else:
                    #this means that the picture was NoneType
                    print("not added to the list: ", i)
                    #skip iteration
                    continue
        i+=1 